In [40]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,8 hours 40 mins
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,15 days
H2O_cluster_name:,H2O_from_python_andyg_vutumg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.793 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [41]:
import pandas as pd

In [42]:
stores_train = pd.read_csv('data/stores_train_preprocessed.csv')

In [43]:
stores_train.isnull().sum()

store_id                        0
grunnkrets_id                   0
lat                             0
lon                             0
chain_name                      0
mall_name                       0
revenue                         0
grunnkrets_population         805
district_population            30
distance_from_busstop           0
busstop_importance_level        0
lv1                             0
lv2                             0
lv3                             0
lv4                             0
households_grunnkrets           0
municipality_name               0
area_km2                       30
district_municipality_name      0
dtype: int64

In [44]:
stores_train['grunnkrets_population'] = stores_train['grunnkrets_population'].fillna(stores_train.grunnkrets_population.mean())
stores_train['district_population'] = stores_train['district_population'].fillna(stores_train.district_population.mean())
stores_train['area_km2'] = stores_train['area_km2'].fillna(stores_train.area_km2.mean())

In [45]:
stores_train.isnull().sum()

store_id                      0
grunnkrets_id                 0
lat                           0
lon                           0
chain_name                    0
mall_name                     0
revenue                       0
grunnkrets_population         0
district_population           0
distance_from_busstop         0
busstop_importance_level      0
lv1                           0
lv2                           0
lv3                           0
lv4                           0
households_grunnkrets         0
municipality_name             0
area_km2                      0
district_municipality_name    0
dtype: int64

In [46]:
data = stores_train

In [47]:
data = data.drop(['store_id','area_km2','mall_name','lat','lon'], axis = 1)

In [48]:
#data.types

In [49]:
stores_train = stores_train[stores_train.revenue > 0.0]

In [50]:
import numpy as np
stores_train['revenue'] = np.log10(stores_train['revenue'])

In [51]:
data = h2o.H2OFrame(data)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [52]:
train, test = data.split_frame(ratios=[0.8], seed = 42)

In [53]:
####################################################################################
#x = train.columns
#y = 'revenue'
#x.remove(y)

In [54]:
model = H2OAutoML(sort_metric='rmsle',
                  max_runtime_secs=7200,
                  seed = 42,
                  max_models = 10,
                  max_runtime_secs_per_model = 720,
                  exclude_algos = ['DeepLearning'],
                  #stopping_metric = 'RMSLE'
                 )

In [55]:
model.train(training_frame = train, y = 'revenue')

AutoML progress: |█
03:18:50.310: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,5.0706964,0.1668218,5.35174,5.035549,5.023178,4.903823,5.0391917
mean_residual_deviance,107.59683,18.715324,136.36348,89.622154,102.5097350,94.539185,114.94961
mse,107.59683,18.715324,136.36348,89.622154,102.5097350,94.539185,114.94961
null_deviance,496658.88,75986.08,629171.25,454583.75,476583.78,482540.47,440415.2
r2,0.5519780,0.0394674,0.5445992,0.5863526,0.5456924,0.5903091,0.4929369
residual_deviance,222256.14,38297.67,285954.22,188027.28,216500.56,197681.44,223117.19
rmse,10.342733,0.8836725,11.677478,9.466898,10.124709,9.723126,10.721456
rmsle,0.8366879,0.0109544,0.8246871,0.84971,0.8309166,0.8312587,0.8468673


In [56]:
h2o_models = h2o.automl.get_leaderboard(model, extra_columns = "ALL")
h2o_models

model_id,rmsle,rmse,mse,mae,mean_residual_deviance,training_time_ms,predict_time_per_row_ms,algo
StackedEnsemble_AllModels_1_AutoML_12_20221112_31850,0.835409,10.3726,107.59,5.06622,107.59,527,0.017417,StackedEnsemble
StackedEnsemble_BestOfFamily_1_AutoML_12_20221112_31850,0.83887,10.4577,109.363,5.10598,109.363,529,0.014407,StackedEnsemble
GBM_2_AutoML_12_20221112_31850,0.852405,10.4566,109.339,5.16245,109.339,320,0.004423,GBM
XRT_1_AutoML_12_20221112_31850,0.874329,10.8293,117.275,5.42834,117.275,2255,0.011435,DRF
DRF_1_AutoML_12_20221112_31850,0.874897,10.895,118.7,5.45365,118.7,1891,0.010358,DRF
GBM_5_AutoML_12_20221112_31850,nan,10.4847,109.929,5.1776,109.929,255,0.005103,GBM
GBM_1_AutoML_12_20221112_31850,nan,10.7128,114.764,5.37284,114.764,387,0.007946,GBM
GBM_3_AutoML_12_20221112_31850,nan,10.4246,108.672,5.16247,108.672,248,0.005176,GBM
GBM_grid_1_AutoML_12_20221112_31850_model_1,nan,10.5973,112.303,5.27714,112.303,272,0.00465,GBM
GBM_grid_1_AutoML_12_20221112_31850_model_2,nan,10.6993,114.474,5.26873,114.474,466,0.009031,GBM


In [57]:
best_model = model.get_best_model()
print(best_model)

Model Details
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_AllModels_1_AutoML_12_20221112_31850

No summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 66.67571351231501
RMSE: 8.165519794374086
MAE: 4.146747769171244
RMSLE: 0.7427189735998443
Mean Residual Deviance: 66.67571351231501
R^2: 0.7215260666866004
Null degrees of freedom: 10018
Residual degrees of freedom: 10011
Null deviance: 2398884.399526334
Residual deviance: 668023.9736798841
AIC: 70528.8950165898

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 107.5901293976551
RMSE: 10.372566191529225
MAE: 5.066224517388992
RMSLE: 0.835408914373952
Mean Residual Deviance: 107.5901293976551
R^2: 0.551944656567376
Null degrees of freedom: 10338
Residual degrees of freedom: 10330
Null deviance: 2483294.4252651567
Residual deviance: 1112374.3478423562
AIC: 77730.05358283107

Cross-Validation Metrics Summary: 
         

In [58]:
best_model.model_performance(test)

ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 90.037940774946
RMSE: 9.488832424220906
MAE: 4.873425627863311
RMSLE: 0.8262636636824937
Mean Residual Deviance: 90.037940774946
R^2: 0.5931389336357749
Null degrees of freedom: 2519
Residual degrees of freedom: 2512
Null deviance: 557683.2272820995
Residual deviance: 226895.6107528639
AIC: 18510.03269442262

In [59]:
y_pred = model.predict(test)
y_pred

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict
3.80177
9.26562
8.88315
6.47638
5.092
8.0867
13.2816
5.50253
6.52521
4.65454


In [60]:
y_truth = test['revenue']
y_truth

revenue
5.02
12.524
14.137
0
0.339
4.755
16.162
1
27.764
2.963


In [61]:
#y_pred.types

In [62]:
#explain_model = model.explain(frame = test, figsize = (8,6))

In [63]:
from h2o.model.regression import h2o_mean_squared_error

h2o_mean_squared_error(y_truth, y_pred)

90.03794013026419